In [ ]:
import pandas as pd

# حاول قراءة الملفات بمعاملات مختلفة
try:
    fake_df = pd.read_csv('/content/Fake.csv', encoding='latin-1', on_bad_lines='skip')
    true_df = pd.read_csv('/content/True.csv', encoding='latin-1', on_bad_lines='skip')
except:
    try:
        fake_df = pd.read_csv('/content/Fake.csv', encoding='utf-8', on_bad_lines='skip')
        true_df = pd.read_csv('/content/True.csv', encoding='utf-8', on_bad_lines='skip')
    except:
        fake_df = pd.read_csv('/content/Fake.csv', engine='python', on_bad_lines='skip')
        true_df = pd.read_csv('/content/True.csv', engine='python', on_bad_lines='skip')

# تحقق من البيانات
print("أبعاد Fake_df:", fake_df.shape)
print("أبعاد True_df:", true_df.shape)
print("\nأعمدة Fake_df:", fake_df.columns.tolist())
print("أعمدة True_df:", true_df.columns.tolist())

أبعاد Fake_df: (23481, 4)
أبعاد True_df: (21417, 4)

أعمدة Fake_df: ['title', 'text', 'subject', 'date']
أعمدة True_df: ['title', 'text', 'subject', 'date']


In [ ]:

# أضف التصنيفات
fake_df['label'] = 1  # أخبار مزيفة
true_df['label'] = 0  # أخبار حقيقية

# ادمج البيانات
df = pd.concat([true_df, fake_df], ignore_index=True)

# خلط البيانات
df = df.sample(frac=1, random_state=42).reset_index(drop=True)


In [ ]:
from datasets import Dataset, DatasetDict
import numpy as np

# Split the data (80% train, 10% validation, 10% test)
train_df, val_df, test_df = np.split(df.sample(frac=1, random_state=42),
                                     [int(.8*len(df)), int(.9*len(df))])

# Create Hugging Face datasets
datasets = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "validation": Dataset.from_pandas(val_df),
    "test": Dataset.from_pandas(test_df)
})

print("Dataset structure ready!")

/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Dataset structure ready!


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load tokenizer and model
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize function
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

# Apply tokenization
tokenized_datasets = datasets.map(tokenize_function, batched=True)

# Load model for sequence classification (2 classes: Fake vs Real)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

print("Model and tokenizer ready!")

Map:   0%|          | 0/35918 [00:00<?, ? examples/s]

Map:   0%|          | 0/4490 [00:00<?, ? examples/s]

Map:   0%|          | 0/4490 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model and tokenizer ready!


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Define metrics function
# Simple metrics without numpy or scikit-learn
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    correct = sum(l == p for l, p in zip(labels, preds))
    total = len(labels)
    accuracy = correct / total
    return {"accuracy": accuracy}

# Set training arguments


In [ ]:
pip install --upgrade transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 88.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 67.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.4
    Uninstalling tokenizers-0.21.4:
      Successfully uninstalled tokenizers-0.21.4
  Attempting uninstall: transformers
    Found existing installation: transformers 4.55.4
    Uninstalling transformers-4.55.4:
      Successfully uninstalled transformers-4.55.4


In [ ]:
import transformers
print(transformers.__version__)


4.56.0


In [ ]:
import sys
import transformers
print(transformers.__file__)
print(transformers.__version__)


/usr/local/lib/python3.12/dist-packages/transformers/__init__.py
4.56.0


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# تحميل التوكنيزر والموديل الجاهز
model_name = "distilbert-base-uncased"  # ممكن تغيّره لأي موديل مناسب
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# افتراضياً، num_labels=2 لو التصنيف بين Fake و Real


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


NameError: name 'df' is not defined

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"  # تعطيل wandb

from transformers import Trainer, TrainingArguments, AutoTokenizer, AutoModelForSequenceClassification, TrainerCallback

# تحميل موديل خفيف
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# استخدام جزء صغير من البيانات للتجربة فقط
train_dataset = tokenized_datasets["train"].select(range(500))  # أول 500 مثال
eval_dataset = tokenized_datasets["validation"].select(range(100))  # أول 100 مثال

# إعدادات تدريب سريعة
training_args = TrainingArguments(
    output_dir="./news_classifier",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,  # أقل عدد epochs
    weight_decay=0.01,
    logging_dir="./logs",
    do_eval=True,
    eval_steps=50,       # تقييم متكرر
    logging_steps=50,    # logs أسرع
    save_strategy="no"   # عدم حفظ checkpoints
)

# Callback لطباعة التقدم
class ProgressCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            completed = state.global_step
            total = state.max_steps
            percent = (completed / total) * 100 if total is not None else 0
            print(f"Step {completed}/{total} ({percent:.2f}%) — {logs}")

# إنشاء Trainer مع callback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    callbacks=[ProgressCallback]
)

# بدء التدريب
trainer.train()


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss


Step 32/32 (100.00%) — {'train_runtime': 1773.665, 'train_samples_per_second': 0.282, 'train_steps_per_second': 0.018, 'total_flos': 66233699328000.0, 'train_loss': 0.3570324778556824, 'epoch': 1.0}


TrainOutput(global_step=32, training_loss=0.3570324778556824, metrics={'train_runtime': 1773.665, 'train_samples_per_second': 0.282, 'train_steps_per_second': 0.018, 'total_flos': 66233699328000.0, 'train_loss': 0.3570324778556824, 'epoch': 1.0})

In [ ]:
# تقييم النموذج على eval_dataset
metrics = trainer.evaluate()
print("Evaluation metrics:", metrics)


Step 32/32 (100.00%) — {'eval_loss': 0.13407285511493683, 'eval_accuracy': 0.96, 'eval_runtime': 90.9307, 'eval_samples_per_second': 1.1, 'eval_steps_per_second': 0.077, 'epoch': 1.0}
Evaluation metrics: {'eval_loss': 0.13407285511493683, 'eval_accuracy': 0.96, 'eval_runtime': 90.9307, 'eval_samples_per_second': 1.1, 'eval_steps_per_second': 0.077, 'epoch': 1.0}


In [ ]:
# نصوص جديدة
texts = [
    "This news article is fake.",
    "This news article is real."
]

# توكينز النصوص
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

# تمريرها للنموذج
import torch
with torch.no_grad():
    logits = model(**inputs).logits

# الحصول على التصنيفات
predictions = torch.argmax(logits, dim=-1)
print("Predicted labels:", predictions.tolist())


Predicted labels: [1, 1]


In [ ]:
# مجلد الحفظ باسم يدل على إنه Transformer
save_directory = "./transformers_news_classifier"

# حفظ الموديل
model.save_pretrained(save_directory)

# حفظ التوكنيزر
tokenizer.save_pretrained(save_directory)

print(f"Model and tokenizer saved to {save_directory}")


Model and tokenizer saved to ./transformers_news_classifier


In [ ]:
import shutil

# المجلد المحفوظ فيه الموديل
save_directory = "./transformers_news_classifier"

# اسم ملف الـ ZIP النهائي
zip_filename = "transformers_news_classifier.zip"

# ضغط المجلد
shutil.make_archive("transformers_news_classifier", 'zip', save_directory)

print(f"Model folder zipped as {zip_filename}")


Model folder zipped as transformers_news_classifier.zip


In [ ]:
from google.colab import files

files.download(zip_filename)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>